<a href="https://colab.research.google.com/github/aniket-work/llamaindex_study/blob/main/llamaindex_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -qU llama-index datasets pinecone-client openai transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.4/809.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

data = load_dataset('squad', split='train')
data = data.to_pandas()[['id', 'context','title']]
data.drop_duplicates(subset='context',keep='first', inplace=True)
data.head()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

,id,context,title
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",University_of_Notre_Dame
5,5733bf84d058e614000b61be,"As at most other universities, Notre Dame's st...",University_of_Notre_Dame
10,5733bed24776f41900661188,The university is the major seat of the Congre...,University_of_Notre_Dame
15,5733a6424776f41900660f51,The College of Engineering was established in ...,University_of_Notre_Dame
20,5733a70c4776f41900660f64,All of Notre Dame's undergraduate students are...,University_of_Notre_Dame


In [3]:
len(data)

18891

In [5]:
from llama_index import Document
docs = []

for i, row in data.iterrows() :
  docs.append(Document(text=row['context'], doc_id=row['id'], extra_info={'title': row['title']} ))
docs[0]

Document(id_='5733be284776f41900661182', embedding=None, metadata={'title': 'University_of_Notre_Dame'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', start_char_idx=None, end_char_idx=N

In [9]:
from google.colab import drive

drive.mount('/content/store')

Mounted at /content/store


In [43]:

# Read the API key from the file
with open("/content/store/MyDrive/aniket_env.txt", "r") as file:
    api_key = file.read().strip()


In [30]:
import os
os.environ['OPENAI_API_KEY'] = api_key


In [33]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=20)

nodes = parser.get_nodes_from_documents(docs)
nodes[0]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


TextNode(id_='f46479f4-6248-42d7-8575-b583340b7cec', embedding=None, metadata={'title': 'University_of_Notre_Dame'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5733be284776f41900661182', node_type=None, metadata={'title': 'University_of_Notre_Dame'}, hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6')}, hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedl

In [34]:
len(nodes)

18891

In [35]:
# Read the API key from the file
with open("/content/store/MyDrive/aniket_env_pinecone.txt", "r") as file:
    pinecone_api_key = file.read().strip()

In [39]:
import pinecone
os.environ['PINECONE_API_KEY'] = pinecone_api_key
os.environ['PINECONE_ENVIRONMENT'] = 'gcp-starter'

pinecone.init(api_key=os.environ['PINECONE_API_KEY'], environment=os.environ['PINECONE_ENVIRONMENT'])

index_name = 'llama-index-intro'
if index_name  not in pinecone.list_indexes():
  pinecone.create_index(index_name, dimension=1536, metric='cosine')

pinecone_index = pinecone.Index(index_name)

In [42]:
from llama_index.vector_stores import PineconeVectorStore

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)